# PetitRADTRANS Emission Spectrum Retrieval Example
### Written by Evert Nasedkin and Paul Mollière
### Based on Mollière (2020)
https://arxiv.org/abs/2006.09394

Here we will outline the process of setting up a RetrievalConfig object,
which is the class used to set up a petitRadTrans retrieval.
The basic process will always be to set up the configuration, and then pass
it to the Retrieval class to run the retrieval using pyMultinest sampling.
Several standard plotting outputs will also be produced by the retrieval class.

## Getting started
You should already have an installation of petitRadTrans on your machine, together with the opacity data files. Multinest and pymultinest are also required. 
If you installed from gitlab, ensure that you have ran <code>./make.sh</code> in both the petitRADTRANS and petitRADTRANS/retrieval directories, which compiles the fortran code used in the retrieval for rebinning spectra.


In [1]:
# Let's start by importing everything we need.
import sys, os
# To not have numpy start parallelizing on its own
os.environ["OMP_NUM_THREADS"] = "1"
import numpy as np
import matplotlib.pyplot as plt

from petitRADTRANS import Radtrans
from petitRADTRANS import nat_cst as nc

# Import the class used to set up the retrieval.
from petitRADTRANS import RetrievalConfig
from petitRADTRANS import Retrieval

# Import Prior functions, if necessary.
from petitRADTRANS.retrieval.util import gaussian_prior

# Import atmospheric model function
# You could also write your own!
from petitRADTRANS.retrieval.models import emission_model_diseq

In [2]:
# Lets start out by setting up a simple run definition
# We'll add the data AFTER we define the model function below
# Full details of the parameters can be found in retrieval_config.py

# Since our retrieval has already ran before, we'll set the mode to 'evaluate' so we can make some plots.
RunDefinitionSimple = RetrievalConfig(retrieval_name = "hst_example_clear_spec", 
                                      run_mode = "retrieval", # This must be 'retrieval' to run PMN        
                                      AMR = False, # We won't be using adaptive mesh refinement        
                                      plotting = False, # We don't want to output plots for each sample
                                      scattering = False)# We don't want to use scattering (only for emission)

# Let's start with the parameters that will remain fixed during the retrieval
RunDefinitionSimple.add_parameter('Rstar', # name
                                  False,   # is_free_parameter
                                  value = 0.651*nc.r_sun)

# Log of the surface gravity in cgs units.
# The transform_prior_cube argument transforms a random sample in [0,1]
# to the physical units we're using in pRT.
RunDefinitionSimple.add_parameter('log_g',True, 
                            transform_prior_cube_coordinate = \
                            lambda x : 2.+3.5*x)

# Planet radius in cm
RunDefinitionSimple.add_parameter('R_pl', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : ( 0.2+0.2*x)*nc.r_jup_mean)

# Interior temperature in Kelvin
RunDefinitionSimple.add_parameter('Temperature', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : 300.+2000.*x,
                            value=0.0)

# Let's include a grey cloud to make sure that the retrieval works, 
# and doesn't find an opaque cloud for the clear atmosphere.
RunDefinitionSimple.add_parameter("log_Pcloud", True, \
                            transform_prior_cube_coordinate = \
                            lambda x : -6.0+8.0*x,
                            value=0.0)                          


Let's start with a simple model. We'll use an isothermal temperature profile, and use free chemistry.
The basic process for every retrieval will be the same. We need to set up a RetrievalConfig object,
adding in the Data objects that we want to fit, the Parameters and their priors, and all of the extra parameters we can use to customize our retrieval.

## Opacities
PetitRadTrans can be setup to use line by line opacities at R=10^6, or a lower resolution correlated-k mode (which we use here). The names of each species in these lists much match the names in the pRT opacity data files, derived from ExoMol line lists.

In addition to the line-by-line opacities, pRT also accounts for rayleigh scattering, CIA opaticies, and opacities for various cloud species.
If a cloud species is used, it will have a set of parameters added in order to retrieve the cloud abundance.
The parameters added depend on the model chosen. For this example, a parameter is added to scale the equilibrium cloud abundances. If the 'eq' parameter is turned off, parameters for the cloud abundance and the cloud base pressure are added.

In [3]:
RunDefinitionSimple.set_rayleigh_species(['H2', 'He'])
RunDefinitionSimple.set_continuum_opacities(['H2-H2', 'H2-He'])

# Here we setup the line species for a free retrieval, setting the prior bounds with the abund_lim parameter
# The retrieved value is the log mass fraction.
RunDefinitionSimple.set_line_species(["CH4", "H2O", "CO2", "CO_all_iso"], free=True, abund_lim = (-6.0,6.0))

# Model Functions
In addition to setting up a retrieval with a standard mode, you may also want to write your own model to be used in the retrieval setup. This function will be passed as the <code>model_generating_function</code> argument when you <code>add_data</code> to the RetrievalConfig class. Here we will outline a simple, cloud-free transmission spectrum model using free retrieval chemistry and an isothermal P-T profile to demonstrate.

Four parameters are required. A pRT object which is created is from the Radtrans() class in pRT and a dictionary of parameters, which is created and stored by the RetrievalConfig class that we defined above. The remaining three parameters deal with plotting and adaptive mesh refinement, which we won't use in this example.

In general, the built in model functions require cgs units, apart from pressure which is given in bar.

In [4]:
# Now we can define the atmospheric model we want to use
def retrieval_model_spec_iso(pRT_object, \
                             parameters, \
                             PT_plot_mode = False,
                             AMR = False):
    """
    retrieval_model_eq_transmission
    This model computes a transmission spectrum based on free retrieval chemistry
    and an isothermal temperature-pressure profile. 
    
    parameters
    -----------
    pRT_object : object
        An instance of the pRT class, with optical properties as defined in the RunDefinition.
    parameters : dict
        Dictionary of required parameters:
            Rstar : Radius of the host star [cm]
            log_g : Log of surface gravity
            R_pl : planet radius [cm]
            Temperature : Isothermal temperature [K]
            species : Abundances for each species used in the retrieval
            Pcloud : optional, cloud base pressure of a grey cloud deck.
    PT_plot_mode : bool
        Return only the pressure-temperature profile for plotting. Evaluate mode only.
    AMR : 
        Adaptive mesh refinement. Use the high resolution pressure grid around the cloud base.

    returns
    -------
    wlen_model : np.array
        Wavlength array of computed model, not binned to data [um]
    spectrum_model : np.array
        Computed transmission spectrum R_pl**2/Rstar**2
    """
    # Make the P-T profile
    pressures = pRT_object.press/1e6 # internally pRT uses cgs

    # Note how we access the values of the Parameter class objects
    temperatures = parameters['Temperature'].value * np.ones_like(pressures)
       
    # If in evaluation mode, and PTs are supposed to be plotted
    if PT_plot_mode:
        return pressures, temperatures
    
    # Make the abundance profiles
    abundances = {}
    msum = 0.0 # Check that the total massfraction of all species is <1
    for species in pRT_object.line_species:
        spec = species.split('_R_')[0] # Dealing with the exo-k naming.
        abundances[species] = 10**parameters[spec].value * np.ones_like(pressures)
        msum += 10**parameters[spec].value
    if msum > 1.0:
        return None, None
    abundances['H2'] = 0.766 * (1.0-msum) * np.ones_like(pressures)
    abundances['He'] = 0.234 * (1.0-msum) * np.ones_like(pressures)
    
    # Find the mean molecular weight in each layer
    from petitRADTRANS.retrieval.util import calc_MMW
    MMW = calc_MMW(abundances)
    
    # Calculate the spectrum
    pRT_object.calc_transm(temperatures, \
                           abundances, \
                           10**parameters['log_g'].value, \
                           MMW, \
                           R_pl=parameters['R_pl'].value, \
                           P0_bar=0.01,
                           Pcloud = 10**parameters['log_Pcloud'].value)
                           # Keep P0_bar at 100. for now!
                           # Otherwise change maximum pressure
    # Transform the outputs into the units of our data.
    wlen_model = nc.c/pRT_object.freq/1e-4
    spectrum_model = (pRT_object.transm_rad/parameters['Rstar'].value)**2.
    return wlen_model, spectrum_model

In [5]:
# Finally, we associate the model with the data, and we can run our retrieval!
import petitRADTRANS # need to get the name for the example data
path_to_data = "" 
path_to_data = petitRADTRANS.__file__.split('__init__.py')[0] # Default location for the example
RunDefinitionSimple.add_data('HST',   # Simulated HST data
                       path_to_data + \
                       'retrieval_examples/transmission/hst_example_clear_spec.txt', # Where is the data
                       model_generating_function = retrieval_model_spec_iso, # The atmospheric model to use
                       data_resolution = 60, # The spectral resolution of the data
                       model_resolution = 120) # The resolution of the c-k tables for the lines

# In general, it is also useful to use the data_resolution and model_resolution arguments.
# The data_resolution argument will convolve the generated model spectrum by the instrumental 
# resolution prior to calculating the chi squared value.
#
# The model_resolution function uses exo-k to generate low-resolution correlated-k opacity tables
# in order to speed up the radiative transfer calculations. 
# We recommend choosing a model_resolution of about 2x the data_resolution.



## Plotting
Let's set up some plotting details so that we can generate nice outputs.

Each parameter can be added to the corner plot, its label changed, and the values transformed to more nature units (eg, the planet radius in jupiter radii, rather than cm).
We can also set the bounds and scaling on the best fit spectrum plot and the limits for the P-T profile plot.
With this complete, our retrieval is ready to go.

Most parameters include a default setting, so the plots will be created even if you don't set any plotting parameters, but the outputs might not be very informative. In general, the possible arguments to `plot_kwargs` follow the naming conventions of matplotlib arguments and functions, with some additions.
Full details of the plotting can be found in the Retrieval class in retrieval.py.

In [6]:
##################################################################
# Define what to put into corner plot if run_mode == 'evaluate'
##################################################################
RunDefinitionSimple.parameters['R_pl'].plot_in_corner = True
RunDefinitionSimple.parameters['R_pl'].corner_label = r'$R_{\rm P}$ ($\rm R_{Jup}$)'
RunDefinitionSimple.parameters['R_pl'].corner_transform = lambda x : x/nc.r_jup_mean
RunDefinitionSimple.parameters['log_g'].plot_in_corner = True
RunDefinitionSimple.parameters['log_g'].corner_ranges = [2., 5.]
RunDefinitionSimple.parameters['log_g'].corner_label = "log g"
RunDefinitionSimple.parameters['Temperature'].plot_in_corner = True
RunDefinitionSimple.parameters['Temperature'].corner_label ="Temp"

# Adding all of the chemical species in our atmosphere to the corner plot
for spec in RunDefinitionSimple.line_species:
    if 'all_iso' in spec:
        # CO is named CO_all_iso, watch out
        RunDefinitionSimple.parameters[spec].corner_label = 'CO'
    RunDefinitionSimple.parameters[spec].plot_in_corner = True
    RunDefinitionSimple.parameters[spec].corner_ranges = [-7,-1.0]

    # Note, the post_equal_weights file has the actual abundances.
    # If the retrieval is rerun, it will contain the log10(abundances)
    RunDefinitionSimple.parameters[spec].corner_transform = lambda x : np.log10(x)

##################################################################
# Define axis properties of spectral plot if run_mode == 'evaluate'
##################################################################
RunDefinitionSimple.plot_kwargs["spec_xlabel"] = 'Wavelength [micron]'
RunDefinitionSimple.plot_kwargs["spec_ylabel"] = r'$(R_{\rm P}/R_*)^2$ [ppm]'
RunDefinitionSimple.plot_kwargs["y_axis_scaling"] = 1e6 # so we have units of ppm
RunDefinitionSimple.plot_kwargs["xscale"] = 'linear'
RunDefinitionSimple.plot_kwargs["yscale"] = 'linear'
RunDefinitionSimple.plot_kwargs["nsample"] = 10

##################################################################
# Define from which observation object to take P-T
# in evaluation mode (if run_mode == 'evaluate'),
# add PT-envelope plotting options
##################################################################
RunDefinitionSimple.plot_kwargs["take_PTs_from"] = 'HST'
RunDefinitionSimple.plot_kwargs["temp_limits"] = [150, 3000]
RunDefinitionSimple.plot_kwargs["press_limits"] = [1e2, 1e-5]

## Running the retrieval
At this point, we are ready to run the retrieval! All we need to do is pass the RunDefinition we just created to the Retrieval class and call its run method. There are a few additional parameters that can be adjusted, but the defaults should work well for almost all use cases. It's not recommended to try to run a retrieval on your laptop, as it's very computationally expensive. However, the included HST example should be able to be run fairly easily!

Now that the plotting parameters are set up, we can start the retrieval using retrieval.run(). Once the retrieval is complete, we can use the plotAll() function to generate plots of the best fit spectrum, the pressure-temperature profile and the corner plots.

In [12]:
# If you want to run the retrieval, you need to choose a different output directory name,
# due to pRT requirements.
#output_dir = path_to_data + 'retrieval_examples/transmission/'
output_dir = ""

retrieval = Retrieval(RunDefinitionSimple,
                      output_dir = output_dir,
                      sample_spec = False,
                      resume=False)
retrieval.run()
#retrieval.plotAll() # We'll make all the plots individually for now.

Setting up PRT Objects
  Read line opacities of CH4_R_120...
 Done.
  Read line opacities of H2O_R_120...
 Done.
  Read line opacities of CO2_R_120...
 Done.
  Read line opacities of CO_all_iso_R_120...
 Done.

  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
 Done.

Starting retrieval: hst_example_clear_spec



Let's make some plots to see how our retrieval went! We need to start by reading in the results

In [ ]:

sample_dict, parameter_dict = retrieval.getSamples()
samples_use = sample_dict[retrieval.retrieval_name]
parameters_read = parameter_dict[retrieval.retrieval_name]


In [ ]:
# Plotting the best fit spectrum
fig,ax,ax_r = retrieval.plotSpectra(samples_use,parameters_read)
plt.show()

In [ ]:
# Plotting the PT profile
fig,ax =retrieval.plotPT(sample_dict,parameters_read)
plt.show()

In [ ]:
# Corner plot
retrieval.plotCorner(sample_dict,parameter_dict,parameters_read)
plt.show()

## Emission Spectrum Example
This retrieval is based on Mollière 2020 for HR8799e, and shows a more realistic example of how to set up a retrieval.

For this example, the module poor_mans_noneq_chemistry is used to solve disequilibrium chemistry, and teh cloud_cond module is used for cloud condensation.
You can import other models from models.py, or write your own function, ensuring that it takes in the same set of inputs and outputs, which are documented in models.py.

This model uses a simple adaptive mesh refinement (AMR) to improve the accuracy around the location of the cloud bases. This method will increase the size of the 

In [ ]:
##########################
# Define the pRT run setup
##########################

# retrieve.py expects this object to be called RunDefinition.
RunDefinition = RetrievalConfig(retrieval_name = "HR8799e_example", # Give a useful name for your retrieval 
                                run_mode = "retrieval", # 'retrieval' to run, or 'evaluate' to make plots
                                AMR = True,             # Adaptive mesh refinement, slower if True
                                plotting = False,       # Only turn on to check your retrieval is working
                                scattering = True)      # Add scattering for emission spectra clouds

## Data and Models
The data can be stored in either a text file (.txt, .dat) or in a .fits file.
If reading a text file, the first column must be the wavelength in micron, the second column the flux in W/m^2/micron for an emission spectrum, or in R_p^2/R_star^2 for a transmission spectrum. The final column must be the error on these data points.
If using a .fits file, it should contain a single HDU object named "SPECTRUM", with three fields: "WAVELENGTH", "FLUX" and "COVARIANCE".

The model_generating_function will take in all of the various retrieval parameters, and output the wavelength and spectrum as calculated by pRT. Several of these function are stored in models.py, but any function that takes the same arguments and returns the same outputs can be used. Later we will show an example of how to write such a function.

For this example we include the GRAVITY data as published in Mollière 2020. To reproduce the published results, be sure to include the archival SPHERE and GPI data from Zurlo et al. 2015 and Greenbaum 2016.

In [ ]:
##################
# Read in Data
##################
path_to_data = "" # default location is petitRADTRANS.__file__.split('__init__.py')[0]
RunDefinition.add_data('GRAVITY',                                        # Name of your dataset, typically the instrument name
                       path_to_data + 'retrieval_examples/emission/HR8799e_Spectra.fits',              # File where the data is stored
                       model_generating_function = emission_model_diseq) # The atmospheric model used in this retrieval


## Parameters and Priors
Here we add all of the parameters used in the retrieval. 
Each parameter must be given a name, which matches the name used in the model function.
Parameters can be set to fixed or free. Fixed parameters must be given a value, while free parameters are given a function that transforms the unit hypercube generated by multinest into physical prior space. Various prior functions are stored in priors.py.

In [ ]:
#################################################
# Add parameters, and priors for free parameters.
#################################################

# This run uses the model of Molliere (2020) for HR8799e
# The lambda function provide uniform priors.

# Distance to the planet in cm
RunDefinition.add_parameter(name = 'D_pl', free = False, value = 41.2925*nc.pc)

# Log of the surface gravity in cgs units.
RunDefinition.add_parameter('log_g',True, 
                            transform_prior_cube_coordinate = \
                            lambda x : 2.+3.5*x)

# Planet radius in cm
RunDefinition.add_parameter('R_pl', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : ( 0.7+1.3*x)*nc.r_jup_mean)

# Interior temperature in Kelvin
RunDefinition.add_parameter('T_int', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : 300.+2000.*x,
                            value=0.0)

# Spline temperature structure parameters. T1 < T2 < T3
# As these priors depend on each other, they are implemented in the model function
RunDefinition.add_parameter('T3', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : x,
                            value=0.0)
RunDefinition.add_parameter('T2', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : x,
                            value=0.0)
RunDefinition.add_parameter('T1', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : x)
# Optical depth model
# power law index in tau = delta * press_cgs**alpha
RunDefinition.add_parameter('alpha', True, \
                            transform_prior_cube_coordinate = \
                            lambda x :1.0+x)
# proportionality factor in tau = delta * press_cgs**alpha
RunDefinition.add_parameter('log_delta', True, \
                            transform_prior_cube_coordinate = \
                            lambda x : x) 
# Chemistry
# A 'free retrieval' would have each line species as a parameter
# Using a (dis)equilibrium model, we only supply bulk parameters.
# Carbon quench pressure
RunDefinition.add_parameter('log_pquench',True,\
                            transform_prior_cube_coordinate = \
                            lambda x : -6.0+9.0*x)
# Metallicity
RunDefinition.add_parameter('Fe/H',True,\
                            transform_prior_cube_coordinate = \
                            lambda x : -1.5+3.0*x)
# C/O ratio
RunDefinition.add_parameter('C/O',True,\
                            transform_prior_cube_coordinate = \
                            lambda x : 0.1+1.5*x)
# Clouds
# Based on an Ackermann-Marley (2001) cloud model
# Width of particle size distribution
RunDefinition.add_parameter('sigma_lnorm', True,
                            transform_prior_cube_coordinate = \
                            lambda x : 1.05 + 1.95*x) 
# Vertical mixing parameters
RunDefinition.add_parameter('log_kzz',True,\
                            transform_prior_cube_coordinate = \
                            lambda x : 5.0+8.*x) 
# Sedimentation parameter
RunDefinition.add_parameter('fsed',True,\
                            transform_prior_cube_coordinate = \
                            lambda x : 1.0 + 10.0*x)
# Data scaling
# If the luminosity of datasets does not agree, we can scale one or more of the datasets
# by a constant factor. 
# Scaling factor name must be the name of the Data class for
# the instrument, + "_scale_factor"                            
#RunDefinition.add_parameter('SPHERE_scale_factor',True,\
#                            transform_prior_cube_coordinate = \
#                            lambda x : 0.8 + 0.4 *x)

In [ ]:
#######################################################
# Define species to be included as absorbers
#######################################################
RunDefinition.set_rayleigh_species(['H2', 'He'])
RunDefinition.set_continuum_opacities(['H2-H2', 'H2-He'])
RunDefinition.set_line_species(['CH4', 'H2O', 'CO2', 'HCN', 'CO_all_iso', 'H2', 'H2S', 'NH3', 'PH3', 'Na', 'K'])
RunDefinition.add_cloud_species('Fe(c)_cd',eq = True,abund_lim = (-3.5,4.5))
RunDefinition.add_cloud_species('MgSiO3(c)_cd',eq = True,abund_lim = (-3.5,4.5))



In [ ]:
retrieval = Retrieval(RunDefinition,
                      output_dir = "",
                      sample_spec = False)


## Plotting
However, before running the retrieval, we should set up the plots.


In [ ]:
##################################################################
# Define what to put into corner plot if run_mode == 'evaluate'
##################################################################
RunDefinition.parameters['R_pl'].plot_in_corner = True
RunDefinition.parameters['R_pl'].corner_label = r'$R_{\rm P}$ ($\rm R_{Jup}$)'
RunDefinition.parameters['R_pl'].corner_transform = lambda x : x/nc.r_jup_mean
RunDefinition.parameters['log_g'].plot_in_corner = True
RunDefinition.parameters['log_g'].corner_ranges = [2., 5.]
RunDefinition.parameters['log_g'].corner_label = "log g"
RunDefinition.parameters['fsed'].plot_in_corner = True
RunDefinition.parameters['log_kzz'].plot_in_corner = True
RunDefinition.parameters['log_kzz'].corner_label = "log Kzz"
RunDefinition.parameters['C/O'].plot_in_corner = True
RunDefinition.parameters['Fe/H'].plot_in_corner = True
RunDefinition.parameters['log_pquench'].plot_in_corner = True
RunDefinition.parameters['log_pquench'].corner_label = "log pquench"

for spec in RunDefinition.cloud_species:
    cname = spec.split('_')[0]
    RunDefinition.parameters['log_X_cb_'+cname].plot_in_corner = True
    RunDefinition.parameters['log_X_cb_'+cname].corner_label = cname

##################################################################
# Define axis properties of spectral plot if run_mode == 'evaluate'
##################################################################
RunDefinition.plot_kwargs["spec_xlabel"] = 'Wavelength [micron]'

RunDefinition.plot_kwargs["spec_ylabel"] = "Flux [W/m2/micron]"
RunDefinition.plot_kwargs["y_axis_scaling"] = 1.0
RunDefinition.plot_kwargs["xscale"] = 'log'
RunDefinition.plot_kwargs["yscale"] = 'linear'
RunDefinition.plot_kwargs["resolution"] = 100.# Maximum resolution, will rebin the data
RunDefinition.plot_kwargs["nsample"] = 100 # If we want a plot with many sampled spectra

##################################################################
# Define from which observation object to take P-T
# in evaluation mode (if run_mode == 'evaluate'),
# add PT-envelope plotting options
##################################################################
RunDefinition.plot_kwargs["take_PTs_from"] = 'GRAVITY'
RunDefinition.plot_kwargs["temp_limits"] = [150, 3000]
RunDefinition.plot_kwargs["press_limits"] = [1e2, 1e-5]

In [ ]:
retrieval = Retrieval(RunDefinition,
                      output_dir = "",
                      sample_spec = False)
retrieval.run()


In [ ]:
retrieval.plotAll()

### Contact
If you need any additional help, don't hesitate to contact Evert Nasedkin (nasedkinevert@gmail.com).